In [3]:
import tensorflow as tf
import numpy as np
import importlib as imp

In [ ]:
# # Meta-training
# $ python main.py \
#   --gpu_id 0 \
#   --savedir "./results/cifar/taml" --id_dataset 'cifar' --ood_dataset 'svhn' \
#   --mode 'meta_train' --metabatch 4 --n_steps 5 --way 5 --max_shot 50 --query 15 \
#   --n_train_iters 50000 --meta_lr 1e-3 \
#   --alpha_on --omega_on --gamma_on --z_on
# Meta-testing
# $ python main.py \
#   --gpu_id 0 \
#   --savedir "./results/cifar/taml" --id_dataset 'cifar' --ood_dataset 'svhn' \
#   --mode 'meta_test' --metabatch 4 --n_steps 10 --way 5 --max_shot 50 --query 15 \
#   --n_test_episodes 1000 \
#   --alpha_on --omega_on --gamma_on --z_on --n_mc_samples 10

In [ ]:
class Arg():
    def __init__(self):
        self.mode = 'meta_train'
        self.savedir = "./results/cifar/taml"
        self.id_dataset = 'cifar'
        self.ood_dataset = 'svhn'
        self.n_train_iters = 100
        self.meta_lr = 1e-3
        self.alpha_on = True
        self.omega_on
